In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from sympy import diff, symbols


In [14]:
tr_data = pd.read_csv('./../data/mnist_train.csv')
ts_data = pd.read_csv('./../data/mnist_test.csv')

In [15]:
tr_data.describe()


,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
count,60000.000000,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,...,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.0000,60000.0,60000.0,60000.0,60000.0
mean,4.453933,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.200433,0.088867,0.045633,0.019283,0.015117,0.0020,0.0,0.0,0.0,0.0
std,2.889270,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.042472,3.956189,2.839845,1.686770,1.678283,0.3466,0.0,0.0,0.0,0.0
min,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0
25%,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0
50%,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0
75%,7.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0
max,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254.000000,254.000000,253.000000,253.000000,254.000000,62.0000,0.0,0.0,0.0,0.0


In [16]:
train_data = tr_data.drop(columns='label').values
train_labels = tr_data.label.values
test_data = ts_data.drop(columns='label').values
test_labels = ts_data.label.values

In [19]:
print(train_data.shape)
print(train_labels.shape)

(60000, 784)
(60000,)


In [21]:
train_data = train_data/255
test_data = test_data/255

In [851]:
class Activation:
    def softmax(self, z):
        return 1/sum(np.exp(z)) * np.exp(z)

    def stablesoftmax(x):
        """Compute the softmax of vector x in a numerically stable way."""
        shiftx = x - np.max(x)
        exps = np.exp(shiftx)
        return exps / np.sum(exps)
    
    def relu(self, z):
        return np.maximum(0, z)

    def tanh(self, z):
        return np.tanh(z)
    
    def sigmoid(self, z):
        return 1/(1 + np.exp(-z))
    
    def use(self, string):
       match string:
        case 'relu':
            return self.relu
        case 'sigmoid':
            return self.sigmoid
        case 'softmax':
            return self.softmax
        case 'tanh':
            return self.tanh
        case 'stablesoftmax':
            return self.stablesoftmax

In [185]:
# activation = Activation().use('softmax')
# print(activation(np.array([-1,3,4])))

In [398]:
# class DenseLayer:
#     def __init__(self, units, a_in, activation=None):
#         self.units = units 
#         self.a_in = a_in
#         self.activation = activation
#         self.w = np.zeros(shape=(a_in[1], units)) #!needs modification
#         self.b = np.zeros(units)

#     def __str__(self):
#         return f'''
#         Number of units: {self.units}
#         Input shape(shape): {self.a_in}
#         weight has shape(w.shape): {self.w.shape}
#         bias has shape(b.shape): {self.b.shape}
#         '''

#     def output(self, a_in):
#         # if(self.activation == None): return a_in@self.w + self.b
#         return self.activation(a_in@self.w + self.b)

        

In [175]:
x = np.array([200, 17])
layer = DenseLayer(2, (None, 2), activation)
print(layer)
print(layer.output(x))


        Number of units: 2
        Input shape(shape): (None, 2)
        weight has shape(w.shape): (2, 2)
        bias has shape(b.shape): (2,)
        
[1.00000000e+00 3.34279552e-80]


In [852]:
class LossFunctions:
    
    def mean_squared_error(y_true, y_pred):
        mse = np.mean((y_true-y_pred)**2)
        return mse
    
    def binary_cross_entropy(y_true, y_pred):
        epsilon = 1e-15
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        return - (y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    
    def softmax_cross_entropy(y_true, y_pred):
        epsilon = 1e-15
        # y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        ce = -np.sum(y_true * np.log(y_pred), axis=1)
        return ce
        

In [853]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, activation):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.activation = [Activation().use(activation[0]), Activation().use(activation[1])]
        
        np.random.seed(3)
        
        self.weights1 = 0.01 * np.random.randn(self.input_size, self.hidden_size)
        self.biases1 = np.zeros((1, self.hidden_size))
        self.weights2 = 0.01 * np.random.randn(self.hidden_size, self.output_size)
        self.biases2 = np.zeros((1, self.output_size))
        
    def forward(self, X):
        self.hidden_layer = self.activation[0](np.dot(X, self.weights1) + self.biases1)
        self.output_layer = self.activation[1](np.dot(self.hidden_layer, self.weights2) + self.biases2)
        return self.output_layer
    
    def backward(self, X, y, learning_rate, loss):
            
        self.weights2 -= learning_rate * 
        self.biases2 -= learning_rate * 
        self.weights1 -= learning_rate * 
        self.biases1 -= learning_rate * 
        
    def train(self, X, y, epochs, learning_rate):
        for i in range(epochs):
            output = self.forward(X)
            loss = LossFunctions.softmax_cross_entropy(y, output)
            self.backward(X, y, learning_rate, loss)
            if i % 10 == 0:
                print("Epoch ", i, " loss: ", np.mean(loss))
                
    def predict(self, X):
        output = self.forward(X)
        predictions = np.argmax(output, axis=1)
        return predictions

    def loss_derivative(self, y_true, y_pred):
        return y_pred - y_true

In [807]:
y_train_one_hot = np.zeros((train_labels.shape[0], 10))
# one-hot encoding the labels into form [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
for i in range(len(train_labels)):
    y_train_one_hot[i, train_labels[i]] = 1

In [808]:
y_train_one_hot.shape

(60000, 10)

In [809]:
y_train_one_hot[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

In [854]:
model = NeuralNetwork(input_size=784, hidden_size=128, output_size=10, activation=['relu', 'softmax'])

In [855]:
model.train(train_data, y_train_one_hot, epochs=1, learning_rate=0.08)

Epoch  0  loss:  11.002100641975726


In [856]:
y_pred = model.predict(test_data)

In [857]:
accuracy = np.mean(y_pred == test_labels)
print("Accuracy: ", accuracy)

Accuracy:  0.098


In [658]:
#  highest -> 17.65%

In [778]:
# model.output_layer.shape

In [850]:
y_pred[2000]

0